In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
train_dir = '1train'
test_dir = '1test'

In [3]:
input_shape = (224, 224, 3)
batch_size = 32
epochs = 80
num_classes = 7

In [20]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [21]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

Found 5830 images belonging to 7 classes.


In [22]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 1467 images belonging to 7 classes.


In [23]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)

In [24]:
for layer in base_model.layers:
    layer.trainable = False

In [25]:
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

In [26]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [28]:
checkpoint = ModelCheckpoint('mobilenet_emotion_model2.keras', monitor='val_accuracy', save_best_only=True, mode='max')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [29]:
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=epochs,
    callbacks=[checkpoint, early_stopping]
)

Epoch 1/6
183/183 ━━━━━━━━━━━━━━━━━━━━ 215s 1s/step - accuracy: 0.2165 - loss: 7.8691 - val_accuracy: 0.2461 - val_loss: 1.9110
Epoch 2/6
183/183 ━━━━━━━━━━━━━━━━━━━━ 205s 1s/step - accuracy: 0.2535 - loss: 1.8997 - val_accuracy: 0.2515 - val_loss: 1.8768
Epoch 3/6
183/183 ━━━━━━━━━━━━━━━━━━━━ 196s 1s/step - accuracy: 0.2515 - loss: 1.8794 - val_accuracy: 0.2461 - val_loss: 1.8652
Epoch 4/6
183/183 ━━━━━━━━━━━━━━━━━━━━ 194s 1s/step - accuracy: 0.2486 - loss: 1.8584 - val_accuracy: 0.2461 - val_loss: 1.8527
Epoch 5/6
183/183 ━━━━━━━━━━━━━━━━━━━━ 195s 1s/step - accuracy: 0.2641 - loss: 1.8447 - val_accuracy: 0.2461 - val_loss: 1.8437
Epoch 6/6
183/183 ━━━━━━━━━━━━━━━━━━━━ 198s 1s/step - accuracy: 0.2501 - loss: 1.8356 - val_accuracy: 0.2461 - val_loss: 1.8374


In [30]:
model.save('final_mobilenet_emotion_model11.keras')